In [9]:
from django.shortcuts import render
from django.http import HttpResponse
from datetime import datetime, date
import logging
from typing import List, Tuple, Dict, Optional
import time
import re
import pymysql
import requests
import json
import pprint
import Dict

import csv

logging.basicConfig(level=logging.INFO)

appname = 'djangoo_api'

username = "pdtpatrick"
password = "u3!WL2uC0dxu"
# in seconds
start_time = 3600 * 48  # 48 hours
# airport code
# airport = "KSEA" # Frankfurt
airport = "EDDF"  # Frankfurt

# Create your views here.
def read_airport(filename: str) -> Dict[str, str]:
    keys = [
        "id",
        "name",
        "city",
        "country",
        "IATA",
        "ICAO",
        "latitude",
        "longitude",
        "altitude",
        "timezone",
        "dst",
        "tz",
        "type",
        "source",
    ]
    airports = csv.DictReader(open(filename), delimiter=",", quotechar='"', fieldnames=keys)
    d = {airport["ICAO"]: airport for airport in airports}
    return d

def call_api(airport: str = None) -> Dict[str, str]:
    """Call opensky API and return all departures

    begin = now - days ago
    end = now
    """
    current_time = time.time()
    URL = f"https://opensky-network.org/api/flights/departure?airport={airport}&begin={int(current_time) - start_time}&end={int(current_time)}"
    logging.info(f"URL is now: {URL}")
    r = requests.get(URL, auth=(username, password))
    if r.status_code == 404:
        logging.error("Cannot find data")
        return None
    assert len(r.json()) != 0
    return r.json()

def process_coordinates(start_time: int, end_time: int) -> List[Dict[str, str]]:
    """Process Coordinates
    Pull data from opensky api, read the csv and create an output like:

    List[Dict[Dict[str, str]]]

    Meaning, we'll have a List[Airport[Coordinates[longitude, latitude]]]
    """
    
    with open('airports.csv') as csv_file:
        data = csv.reader(csv_file, delimiter=',')
        out_data=[]
        
        for departure in call_api(read_airport):
            arrival_airport = departure["estarrival"]
            departure_airport = departure["estdeparture"]
            
            # departure
            long_departure = departure_airport["Longitude"]
            lat_departure = departure_airport["Latitude"]
            
            out_data.append(departure_airport[
              {
                   "Longitude":long_departure, 
                   "Latitude": lat_departure
                }  
            ]   
            )  
    return json.dumps(out_data)

ModuleNotFoundError: No module named 'Dict'